<a href="https://colab.research.google.com/github/ashikshafi08/Learning-Fastai/blob/main/Chapters%20Notebooks/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter - 5 (Fastai Book)**

# Image Classification 



In [ ]:
# Run once

!pip install fastai --upgrade

In [1]:
# Certain utils

!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *

# Needed packages

from fastai import *
from fastai.vision.all import *

Mounted at /content/gdrive


As in previous lectures we peeked under the hood, but the important thing is to make our model perform really well we have to keep an eye out for lot of details. 

This process requires being able to look inside the neural network as it trains and as it make predictions, find possible problems and know how to fix them

From now on will dive deep into mechanics of deep learning, things like: 
- Architecture of computer vision model , NLP , tabular model.
- how to create arch that matches the needs of our particular domain.
- how to get the best possible results from the training process.
- and how to make things faster
- what changes we gotta make as the datasets change etc...

Going to make the basic application looked earlier in the Chapter 1, but will do 2 things to that : 

- Make them better
- Apply them to a wider variety of types of data

Well to work out these 2 things, we gotta learn the pieces of deep learning puzzle, which includes:

- Types of layers
- Regularization methods
- optimizers
- how to put layers together into architectures
- labelling techniques etc...

## **From Dogs and Cats to Pet Breeds**

But in Chapter 1 we discussed about Dog v/s Cat which was easy enough to work on, but it seems this `PETs` dataset is more complex than what we did in the Chapter 1. Where now will dive deep to figure out what breed of pet is shown in each image.

In practical things will be more different like, 

- we start with a dataset we know nothing about
- figure out how it is put together
- how to extract the data we needed from it
- and what the data looks like

In [2]:
# Getting the data 
path = untar_data(URLs.PETS)

To get those breeds out of the data, firstly we gotta understand how this data is laid out. Such details of data layout are a vital piece of deep learning puzzle. 

Data is usually provided in one of these 2 ways: 

- Individual files representing items of data, such as text docs or images which are **organized into folders or with filenames** representing info about those items.
- A table of data (e.g CSV format) in which **each row is an item** and may include **filenames providing connections data in the table and data in other formats**, such as text or images.

But there are exceptions to these above rules, in domains like genomic, where there will be a **binary database or even network streams** but overall the vast majority of datasets will be in the **combinations of the above 2 rules**.

In [5]:
# To see what's in our dataset 

path.ls()

(#2) [Path('/root/.fastai/data/oxford-iiit-pet/annotations'),Path('/root/.fastai/data/oxford-iiit-pet/images')]

Well this dataset provides us with `images` and `annotations`, the **annotation library** --> contains information where the pets are. 


* **classification** --> what the pets are 
* **localization** --> where the pets are 

In this chapter will work on **classification** and ignore the annotations directory for now. 

In [6]:
# Looking into the image directory 

(path /'images').ls()

(#7393) [Path('/root/.fastai/data/oxford-iiit-pet/images/leonberger_31.jpg'),Path('/root/.fastai/data/oxford-iiit-pet/images/samoyed_149.jpg'),Path('/root/.fastai/data/oxford-iiit-pet/images/Russian_Blue_221.jpg'),Path('/root/.fastai/data/oxford-iiit-pet/images/american_bulldog_74.jpg'),Path('/root/.fastai/data/oxford-iiit-pet/images/shiba_inu_64.jpg'),Path('/root/.fastai/data/oxford-iiit-pet/images/scottish_terrier_71.jpg'),Path('/root/.fastai/data/oxford-iiit-pet/images/chihuahua_141.jpg'),Path('/root/.fastai/data/oxford-iiit-pet/images/english_setter_21.jpg'),Path('/root/.fastai/data/oxford-iiit-pet/images/Maine_Coon_119.jpg'),Path('/root/.fastai/data/oxford-iiit-pet/images/beagle_175.jpg')...]

Most functions and methods in **fastai** that returns a collection (like above) use a *class* called `L` .

Yet this is an enhanced version of the ordinary Python `list` type, for instance when we display a object of this class in a notebook. It appears in the format, 

- first thing that is shown is the number of items in the collection, prefixed with `#`. 
- certain outputs of this list is suffixed with ellipsis, means only first few items are displayed. As we don't want to to see more than 7000 filenames on our screens.
--- 

If we examine these filenames, we can see how they are structured. Each filenames contains the pet breed, then an underscore **(_)** , a **number** and finally the **file extension** (png , jpg etc..).

Now we gotta make a piece of code that extracts the breed from a single **Path**. Lets pick one of the filenames and play with it to find more.



In [8]:
# Testing out how a file name looks like 

fname = (path / 'images').ls()[0]
fname

Path('/root/.fastai/data/oxford-iiit-pet/images/leonberger_31.jpg')

Most powerful and flexible way to extract information from a string like this is to use a **regular expression**, also known as **regex**.

A regular expression is a special string, written in the regular expression language. Which specifies a general rule for deciding whether another **string passes a test** (matche or not) and also **parsing a particular part or parts out of that string**. In our case we need a regular expression that extracts the pet breed from the *filename*